In [ ]:
import os
import sys
from glob import glob
import numpy as np
import cv2

from lib import calib, app, extract, utils, plotting

%load_ext autoreload
%autoreload 2

# Define DATA_DIR

In [ ]:
DATA_ROOT_DIR = '../data'
DATA_DIR = os.path.join(DATA_ROOT_DIR, "2017_12_17", "bottom_post")
# DATA_DIR = os.path.join(DATA_ROOT_DIR, "2017_12_10/bottom")

# Extrinsic calibration
EXTRINSIC_DATA_DIR = os.path.join(DATA_DIR, "extrinsic_calib")
DUMMY_SCENE = os.path.abspath(os.path.join("..", "configs", "dummy_scene.json"))

# Automatically defined params
No need to run this if all you're doing is plotting

In [ ]:
YEAR = 2017 if '2017' in DATA_DIR else 2019

# Camera settings
CAMS = os.listdir(os.path.join(EXTRINSIC_DATA_DIR, "videos"))
CAMS = sorted([int(os.path.splitext(cam)[0][-1]) for cam in CAMS])
print("Cams:", CAMS)

# Intrinsic calibration
INTRINSIC_DATA_DIR = os.path.join(DATA_ROOT_DIR, "intrinsic_calib", str(YEAR))

# 1. Intrinsic Calibration

## 1.1. Extract frames

- `.` - Next frame
- `,` - Previous frame
- `s` - Save current frame
- `q` - Quit

In [ ]:
frames_fpath = os.path.join(INTRINSIC_DATA_DIR, "frames")
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)

intrinsic_video_path = os.path.join(INTRINSIC_DATA_DIR, "videos", "1.mp4")
extract.get_frames(
    vid_fpath=intrinsic_video_path, 
    frame_output_dir=frames_fpath
)

## 1.2. Find calibration board corners

In [ ]:
# Intrinsic checkerboard info
INTRINSIC_BOARD_SHAPE = (9, 6) # (horizontal, vertical) num of inner corners of checkerboard pattern
INTRINSIC_SQUARE_LEN = 0.04# (length of one side of black/white chessboard square in metres)

# prepare the output dir
frames_fpath = os.path.join(INTRINSIC_DATA_DIR, "frames")
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)
    
# 'window_size' sets the size of the calibration board corner detector window size
app.extract_corners_from_images(
    img_dir=frames_fpath, 
    out_fpath=os.path.join(INTRINSIC_DATA_DIR, "points.json"), 
    board_shape=INTRINSIC_BOARD_SHAPE, 
    board_edge_len=INTRINSIC_SQUARE_LEN, 
    window_size=5, 
    remove_unused_images=True
)

## 1.3. Visualise

In [ ]:
app.plot_corners(os.path.join(INTRINSIC_DATA_DIR, "points.json"))

## 1.4. Calibrate

In [ ]:
K, D, R, t, used_points, rms = app.calibrate_fisheye_intrinsics(
    points_fpath=os.path.join(INTRINSIC_DATA_DIR, "points.json"), 
    out_fpath=os.path.join(INTRINSIC_DATA_DIR, "camera.json")
)
print(f"\nRMS Error is {rms:.3f} pixels")

## 1.5. Plot undistorted points using intrinsic parameters

In [ ]:
scene = app.plot_points_fisheye_undistort(
    points_fpath=os.path.join(INTRINSIC_DATA_DIR, "points.json"), 
    camera_fpath=os.path.join(INTRINSIC_DATA_DIR, "camera.json")
)

# 2. Extrinsic Calibration

## 2.1. Extract frames from videos

In [ ]:
# Get paths of the calibration videos
calib_vids = glob(os.path.join(EXTRINSIC_DATA_DIR, "videos", "cam[1-9].mp4"))
print(calib_vids)

### Launch GUI to extract the frames including the calibration board
- `.` - Next frame
- `,` - Previous frame
- `s` - Save current frame
- `q` - Quit

In [ ]:
camera = 1  # Change as needed (in this example we have cameras 1,2,...

frames_fpath = os.path.join(EXTRINSIC_DATA_DIR, "frames", str(camera))
if not os.path.exists(frames_fpath):
    os.makedirs(frames_fpath)
print("The the output folder is", frames_fpath)

extract.get_frames(calib_vids[camera-1], frames_fpath)
# cv2.waitKey(0); # Needed to close window properly

print('GREAT JOB!!')

### Convert frames to negatives

In [ ]:
# Do this only if the checkerboard with a black background was used for extrinsic calibration

for cam in CAMS:
    frames_fpath = os.path.join(EXTRINSIC_DATA_DIR, "frames", str(cam))
    neg_frames_dir = os.path.join(EXTRINSIC_DATA_DIR, "neg_frames", str(cam))
    if not os.path.exists(neg_frames_dir):
        os.makedirs(neg_frames_dir)
    print("The the output folder is", neg_frames_dir)
    
    for fname in os.listdir(frames_fpath):
        if fname.lower().endswith(".jpg") or fname.endswith(".png"):
            img = cv2.imread(os.path.join(frames_fpath, fname))
            img_neg = (255 - img)
            cv2.imwrite(os.path.join(neg_frames_dir, fname), img_neg)

## 2.2. Find calibration board corners
Note: This takes a while!

In [ ]:
# Extrinsic checkerboard info
EXTRINSIC_BOARD_SHAPE = (9, 6)
EXTRINSIC_SQUARE_LEN = 0.088

#Set directories
points_dir = os.path.join(EXTRINSIC_DATA_DIR, 'points')
if not os.path.exists(points_dir):
    os.makedirs(points_dir)
data_dirs = [[
    os.path.join(EXTRINSIC_DATA_DIR, "frames", str(cam)),
    os.path.join(points_dir, f"points{cam}.json")
] for cam in CAMS]

# Find calibration board corners in images and save points
for [img_dir, out_fpath] in data_dirs:
    # 'window_size' sets the size of the calibration board corner detector window size
    app.extract_corners_from_images(
        img_dir, 
        out_fpath,
        EXTRINSIC_BOARD_SHAPE,
        EXTRINSIC_SQUARE_LEN, 
        window_size=5, 
        remove_unused_images=False
    )

### Correct points that were detected in the reversed order

In [ ]:
points_fpaths = glob(os.path.join(EXTRINSIC_DATA_DIR, "points", "points[1-9].json"))
print(points_fpaths)

# list of frames where checkerboard points were detected in the wrong orientation
frames = [[],
         [],
         [],
         [],
         [],
         []]
assert len(frames) == len(points_fpaths)

for i in range(len(frames)):
    if frames[i]:
        points, fnames, board_shape, board_edge_len, cam_res = utils.load_points(points_fpaths[i])
        for f in frames[i]:
            img_name = f"img{f:05}.jpg"
            index = fnames.index(img_name)
            img_pts = points[index]
            points[index] = np.flip(img_pts, (0, 1))
        utils.save_points(points_fpaths[i], points, fnames, board_shape, board_edge_len, cam_res)

## 2.3. Calibrate pairwise extrinsics

In [ ]:
camera_fpaths = [os.path.join(INTRINSIC_DATA_DIR, "camera.json")]*len(CAMS)
points_fpaths =  sorted(glob(os.path.join(EXTRINSIC_DATA_DIR, 'points','points[1-9].json')))
manual_points_fpath = os.path.join(EXTRINSIC_DATA_DIR, "points", "manual_points.json")
manual_points_fpath = manual_points_fpath if os.path.exists(manual_points_fpath) else None
scene_fpath = os.path.join(EXTRINSIC_DATA_DIR, f"{len(points_fpaths)}_cam_scene.json")

app.calibrate_fisheye_extrinsics_pairwise(
    camera_fpaths, points_fpaths, 
    out_fpath=scene_fpath,# cams=CAMS,
    dummy_scene_fpath=DUMMY_SCENE,
    manual_points_fpath=manual_points_fpath # include this only if the calibration needs manual points
)

## 2.4. Run calibration SBA
Note: Also takes a while!

In [ ]:
scene_sba_fpath = scene_fpath.replace('.json','_sba.json')

res = app.sba_board_points_fisheye(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath, 
    manual_points_fpath=manual_points_fpath#, manual_points_only=True,
    # camera_indices=None
)
print(f"Before: mean: {np.mean(res['before'])}, std: {np.std(res['before'])}")
print(f"After: mean: {np.mean(res['after'])}, std: {np.std(res['after'])}")

## 2.5. Visualise (optional)

### Plot checkerboard points

In [ ]:
app.plot_scene(DATA_DIR, dark_mode=True)

### Plot manually defined points

In [ ]:
app.plot_scene(DATA_DIR, manual_points_only=True, dark_mode=True)